In [1]:
import sys

import nltk

import pymysql
import os
import cv2
from PIL import Image
import pytesseract
import requests
import math
import re
from collections import Counter
import fuzzywuzzy.fuzz
nltk.download('punkt')






# Get student USN from command line argument
student_usn = sys.argv[1]

# Connect to the MySQL database
db = pymysql.connect(
    host="localhost",
    user="root",
    password="",
    database="aep"
)
cursor = db.cursor()


# Fetch student's answer records
cursor.execute("SELECT * FROM answer WHERE usn=%s", student_usn)
student_data = cursor.fetchall()


# Function to extract text from an image using OCR
def get_text_from_image(file_to_ocr):
    im = Image.open(file_to_ocr)
    txt = pytesseract.image_to_string(im)
    return txt


# Path to the directory containing student's answer images
directory = os.path.join("MachineLearning\answerimage")
image_path = (r'C:\xampp1\htdocs\WebApp\AEP')


# Loop through the student's data to process images and extract text
for n in range(len(student_data)):
    path = os.path.join(directory, str(student_data[n][0]))
    image_path += str(student_data[n][0])

    if not os.path.exists(path):
        os.mkdir(path)

    for j in range(1, len(student_data[n])):
        if len(student_data[n][j]) != 0:
            image_files = re.split(r' *[\,\?!][\""\)\]]* *', student_data[n][j])
            for img_file in image_files[:-1]:
                img_filename = img_file.split('/')[1].split('.')[0][0] + ".txt"
                txt_path = os.path.join(path, img_filename)
                if os.path.isfile(txt_path):
                    os.remove(txt_path)
            
            for img_file in image_files[:-1]:
                txt = get_text_from_image(os.path.join(image_path, img_file.split('/')[1]))
                img_filename = img_file.split('/')[1].split('.')[0][0] + ".txt"
                with open(os.path.join(path, img_filename), 'a') as f:
                    f.write(txt)


# Function to assign marks based on the length of the answer
def ans_len(num_words, ans_length):
    marks = 0
    if num_words > 250:
        marks = 10
    elif 230 < num_words <= 250:
        marks = 9
    elif 200 < num_words <= 230:
        marks = 8
    elif 180 < num_words <= 200:
        marks = 7
    elif 160 < num_words <= 180:
        marks = 6
    elif 140 < num_words <= 160:
        marks = 5
    elif 110 < num_words <= 140:
        marks = 4
    elif 80 < num_words <= 110:
        marks = 3
    elif 40 < num_words <= 80:
        marks = 2
    elif 3 < num_words <= 40:
        marks = 1
    
    if ans_length >= 15:
        length_marks = 5
    elif 13 <= ans_length < 15:
        length_marks = 4
    elif 10 <= ans_length < 13:
        length_marks = 3
    elif 5 <= ans_length < 10:
        length_marks = 2
    elif 1 < ans_length < 5:
        length_marks = 1
    else:
        length_marks = 0
    
    total_marks = (marks + length_marks) / 2
    return int(total_marks)


# Function to check grammar and spelling errors using TextGears API
def grammar_check(sentence, num_words, ans_length):
    api_key = "your_textgears_api_key"
    req = requests.get(f"https://api.textgears.com/check.php?text={sentence}&key={api_key}")
    errors = req.json()['errors']
    count_spelling = sum(1 for error in errors if error['type'] == 'spelling')
    count_grammar = sum(1 for error in errors if error['type'] == 'grammar')

    error_rate1 = count_grammar / ans_length
    error_rate2 = count_spelling / num_words
    percentage_of_error = int(math.sqrt(error_rate1) + math.sqrt(error_rate2) * 10)
    return percentage_of_error


# Function to preprocess the sentence (remove punctuation, stop words, etc.)
import string


def preprocess_sentence(stmt):
    stmt = stmt.lower()
    remove = dict.fromkeys(map(ord, '\n' + string.punctuation), " ")
    stmt = stmt.translate(remove)
    
    from nltk.corpus import stopwords
    from nltk.tokenize import word_tokenize
    
    stop_words = set(stopwords.words('english'))
    word_tokens = word_tokenize(stmt)
    words = [w for w in word_tokens if w not in stop_words]

    from nltk import pos_tag
    
    processed_sentence = " ".join([word for word in words if pos_tag([word])[0][1] != 'IN'])
    return processed_sentence



# Function to match student's answer with the key answer
def match_answers_with_key(Sans1, Kans1):
    from nltk.tokenize import sent_tokenize

    answer = "\n".join(preprocess_sentence(stmt) for stmt in sent_tokenize(Sans1))
    key = "\n".join(preprocess_sentence(stmt) for stmt in sent_tokenize(Kans1))

    def synonym_exists(word, answer_tokens):
        synonyms = {lemma.name() for syn in wordnet.synsets(word) for lemma in syn.lemmas()}
        return any(synonym in answer_tokens for synonym in synonyms)
    
    answer_tokens = set(word_tokenize(answer))
    key_sentences = key.split('\n')

    key_match_ratio = max(
        sum(1 for sentence in key_sentences if sentence in answer) / len(key_sentences) * 100,
        sum(1 for word in set(word_tokenize(key)) if word in answer_tokens or synonym_exists(word, answer_tokens)) / len(set(word_tokenize(key))) * 100
    )

    return key_match_ratio

# Additional functions and evaluation logic would follow the same pattern.

# Train Part

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import accuracy_score, classification_report

def train_model(answer_lengths, error_rates, key_match_ratios, labels):
    # Define the model architecture
    model = RandomForestClassifier(n_estimators=100, random_state=42)
    
    # Train the model
    X = np.array([answer_lengths, error_rates, key_match_ratios]).T
    y = np.array(labels)
    model.fit(X, y)
    
    # Evaluate the model's performance
    y_pred = model.predict(X)
    accuracy = accuracy_score(y, y_pred)
    print("Accuracy:", accuracy)
    print("Classification Report:")
    print(classification_report(y, y_pred))

# Example usage
answer_lengths = [...]  # list of answer lengths
error_rates = [...]  # list of error rates
key_match_ratios = [...]  # list of key match ratios
labels = [...]  # list of labels (e.g., 0 for incorrect, 1 for correct)
train_model(answer_lengths, error_rates, key_match_ratios, labels)

# Test Part

def process_student_answer(student_data, image_path):
    # Extract text from images using OCR
    texts = []
    for img_file in student_data:
        txt = get_text_from_image(os.path.join(image_path, img_file.split('/')[1]))
        texts.append(txt)
    
    # Preprocess the extracted text
    preprocessed_texts = [preprocess_sentence(txt) for txt in texts]
    
    # Calculate the length of the answer
    answer_lengths = [len(txt.split()) for txt in preprocessed_texts]
    
    # Check grammar and spelling errors using the TextGears API
    error_rates = [grammar_check(txt, len(txt.split()), len(txt.split())) for txt in preprocessed_texts]
    
    # Match the student's answer with the key answer
    key_match_ratios = [match_answers_with_key(txt, key_answer) for txt in preprocessed_texts]
    
    return answer_lengths, error_rates, key_match_ratios

# Example usage
student_data = [...]  # list of image files
image_path = 'C:\xampp1\htdocs\WebApp\AEP'
answer_lengths, error_rates, key_match_ratios = process_student_answer(student_data, image_path)




c:\Users\ekted\AppData\Local\Programs\Python\Python312\Lib\site-packages\fuzzywuzzy\fuzz.py:11: UserWarning: Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning
  warnings.warn('Using slow pure-python SequenceMatcher. Install python-Levenshtein to remove this warning')
[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\ekted\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!
